In [2]:
import torch, torchvision
print(torch.__version__)       # 1.13.1
print(torchvision.__version__) # 0.14.1

1.13.1
0.14.1


In [6]:
import Utilities
Utilities.downloadPEALViT()
Utilities.unzipSquareImages()

Downloading...
From (original): https://drive.google.com/uc?id=1EZiws_atuzFLapKYyM9j2668UHSdtAs8
From (redirected): https://drive.google.com/uc?id=1EZiws_atuzFLapKYyM9j2668UHSdtAs8&confirm=t&uuid=a619f2d4-96cb-4738-a414-8821c42385c8
To: /Users/mawy/Desktop/saevision/vit.cpl
100%|██████████| 343M/343M [00:47<00:00, 7.22MB/s]]


[LOG] Downloaded ViT model to /Users/mawy/Desktop/saevision/vit.cpl
[LOG] Extracting Images-2.zip → /Users/mawy/Desktop/Square Images
[LOG] Extracting Images-3.zip → /Users/mawy/Desktop/Square Images
[LOG] Extracting Images-1.zip → /Users/mawy/Desktop/Square Images
[LOG] Extracting Images-4.zip → /Users/mawy/Desktop/Square Images
[LOG] Images ready in '/Users/mawy/Desktop/Square Images' (flattened)


In [7]:
loader = Utilities.createImageDataloader(
    path="/Users/mawy/Desktop/Square Images",
    model_name="vit_b_16",
    image_size=64,
    batch_size=64,
    shuffle=True,
    drop_last=True,
)

In [11]:
import torch
import torch.nn as nn
from torchvision.models.vision_transformer import VisionTransformer

ckpt_path   = "vit.cpl"
device      = "cuda" if torch.cuda.is_available() else "cpu"

state_raw = torch.load(ckpt_path, map_location="cpu")
state = {k.replace("model.", "", 1): v for k, v in state_raw.items()}

model = VisionTransformer(
    image_size = 64,
    patch_size = 16,
    num_layers = 12,
    num_heads  = 12,
    hidden_dim = 768,
    mlp_dim    = 3072,
    dropout    = 0.0,
    num_classes = 0,
)

model.heads = nn.Identity()

model.load_state_dict(state, strict=False)
model = model.to(device).eval()

In [12]:
with torch.inference_mode():
    feats = model(torch.randn(1, 3, 64, 64).to(device))   # (1, 768)
print("Feature-Vektor-Shape:", feats.shape)

Feature-Vektor-Shape: torch.Size([1, 768])


In [13]:
loader_iter = iter(loader)          
batch      = next(loader_iter) 

imgs = batch.to(device)         

with torch.inference_mode():
    feats = model(imgs)         

print(feats.shape)

torch.Size([64, 768])
